# landmark 
    - 웹캠

In [81]:
import cv2
import numpy as np
import dlib

cap = cv2.VideoCapture(0)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

while True:
    _, frame = cap.read()
    frame = cv2.flip(frame, 1)
    # gray scale ( 1가지채널만 있기때문에 연산이 빨라진다 )
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = detector(gray)
    for face in faces:
        # 얼굴의 좌표값이 나온다
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.left()
        # 사각형 그리기
        # cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 3)
        
        # 랜드마크 그리기
        landmarks = predictor(gray, face)
        
        for n in range(0, 68):
            # 포인트의 번호 얻기
            x = landmarks.part(n).x
            y = landmarks.part(n).y 
            cv2.putText(frame, str(n), (x-3,y-7) ,2, 0.4, (255,0, 0))
            cv2.circle(frame, (x, y), 2, (255, 0, 0), -1)
        
            
        #cv2.line(image, 시작, 끝, 색, 두께)
        # 입꼬리
        cv2.line(frame, (landmarks.part(3).x, landmarks.part(3).y) ,
                 (landmarks.part(48).x, landmarks.part(48).y) , (255,0,0), 2)
        cv2.line(frame, (landmarks.part(13).x, landmarks.part(13).y) ,
                 (landmarks.part(54).x, landmarks.part(54).y) , (255,0,0), 2)
        #미간사이
        cv2.line(frame, (landmarks.part(21).x, landmarks.part(21).y) ,
                 (landmarks.part(22).x, landmarks.part(22).y) , (255,0,0), 2)
        #입 위~아래
        cv2.line(frame, (landmarks.part(51).x, landmarks.part(51).y) ,
                 (landmarks.part(57).x, landmarks.part(57).y) , (255,0,0), 2)
        #코 끝
        #눈 위아래 사이
        cv2.line(frame, (landmarks.part(37).x, landmarks.part(37).y) ,
                 (landmarks.part(41).x, landmarks.part(41).y) , (255,0,0), 2)
        cv2.line(frame, (landmarks.part(43).x, landmarks.part(43).y) ,
                 (landmarks.part(47).x, landmarks.part(47).y) , (255,0,0), 2)
        
        
        # 입꼬리        
        cv2.putText(frame, "L-mouth to cheeck : ", (10, 50), 2, 0.4, (0,255,255))
        cv2.putText(frame, str(get_distance(3, 48))[:6], (150, 50), 2, 0.4, (0,255,255))
        cv2.putText(frame, "R-mouth to cheeck : ", (10, 65), 2, 0.4, (0,255,255))
        cv2.putText(frame, str(get_distance(13, 54))[:6], (150, 65), 2, 0.4, (0,255,255))
        # 미간사이
        cv2.putText(frame, "Between eyebrows : ", (10, 80), 2, 0.4, (0,255,255))
        cv2.putText(frame, str(get_distance(21, 22))[:6], (150, 80), 2, 0.4, (0,255,255))
        # 입 위-아래
        cv2.putText(frame, "Mouth size : ", (10, 95), 2, 0.4, (0,255,255))
        cv2.putText(frame, str(get_distance(51, 57))[:6], (150, 95), 2, 0.4, (0,255,255))
        # 눈 위아래 사이
        cv2.putText(frame, "L-eye size : ", (10, 110), 2, 0.4, (0,255,255))
        cv2.putText(frame, str(get_distance(37, 41))[:6], (150, 110), 2, 0.4, (0,255,255))
        cv2.putText(frame, "R-eye size : ", (10, 125), 2, 0.4, (0,255,255))
        cv2.putText(frame, str(get_distance(43, 47))[:6], (150, 125), 2, 0.4, (0,255,255))
        
    
    cv2.imshow("Frame", frame)
    
    key = cv2.waitKey(1)
    if key == 27:
        cap.release()
        cv2.destroyWindow('Frame')
        break


In [39]:
import math
def get_distance(n1, n2):
    return (((landmarks.part(n1).x - landmarks.part(n2).x) ** 2) + ((landmarks.part(n1).y - landmarks.part(n2).y)) ** 2) ** 0.5

# 감정의 종류
## 분노, 놀라움, 혐오, 슬픔, 행복, 공포, 무표정

# 요소종류

## 랜드마크 번호기준
    - 입꼬리 ( 좌 = 3 - 48 ), (우 = 13 - 54)
    - 미간사이 ( 21 - 22)
    - 입 위아래 ( 위 = 51, 아래 = 57 (
    - 코 끝 ( 좌 - 31, 가운데 - 33, 우 - 35 )
    - 눈사이 위아래 사이 ( 좌 - 37,38 - 41,40 ) ( 우 - 43,44 - 47, 46 )

## 감정에 따른 얼굴 형태의 변화
https://www.credible-ss.com/blog/2013/01/07/The-Seven-Basic-Emotions-Do-you-know-them.aspx

# 감정에 따른 표정 변화
    - 분노 : 미간사이가 좁아진다
    - 놀라움 :눈의 위아래가 넓어진다
    - 혐오 : 미간사이가 좁아진다, 코의 양끝이 올라간다
    - 슬픔 : 눈의 위아래가 좁아진다, 입꼬리가 내려간다, 광대가 올라간다
    - 행복 : 입꼬리가 올라간다, 광대가 올라간다, 눈썹이 올라간다, 
    - 공포 : 입을 벌린다, 눈이 커진다, 

## 분노
    - 눈썹이 내려감
    - 눈썹이 가운데로 몰림
    - 입이 약간 벌어짐
    - 코의 양끝이 올라감

## 슬픔
    - 입꼬리가 내려감
    - 약간의 찡그림
    - 눈썹끝이 내려감
    - 눈이 작아짐
    

## 행복
    - 입꼬리가 올라감
    - 입이 약간 벌어짐
    - 눈이 약간 작아짐

## 놀람
    - 눈썹이 올라감
    - 눈의 세로 크기가 커짐
    - 입이 벌어짐

/images/train/
## 얼굴만 따로 저장함

In [23]:
import os, cv2, dlib
import matplotlib.pyplot as plt
import numpy as np

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# angry_path_dir = 'c:/MyJupyter/Image_analysis/images/train/angry/'
image_path_dir = 'c:/MyJupyter/Image_analysis/images/angry/'
save_dir = 'c:/MyJupyter/Image_analysis/mine/'
image_list = os.listdir(image_path_dir)
image_number = 1

for f in image_list[:50]:
    # 하나씩 불러오기
    file = image_path_dir + f
    # 이미지로 만들기
    image = cv2.imread(file)
    
    if np.shape(image) == ():
        continue
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)
    
    # 얼굴탐지 실패시 for 문을 만들어감
    for face in faces:
        # 얼굴의 좌표값이 나온다
        x1 = face.left()
        y1 = face.top()
        x2 = face.right()
        y2 = face.bottom()
        
        
        # 랜드마크 그리기
        landmarks = predictor(gray, face)
        
        for n in range(0, 68):
            # 포인트의 번호 얻기
            x = landmarks.part(n).x
            y = landmarks.part(n).y 
            cv2.putText(gray, str(n), (x-3,y-7) ,2, 0.4, (255,0, 0))
            cv2.circle(gray, (x, y), 2, (255, 0, 0), -1)
        
                
        cut = gray.copy()
        cut = cut[y1:y2, x1:x2]
        save = save_dir + str(image_number) + ".jpg"
        image_number += 1
        cv2.imwrite(save, cut)
            